# Tutorial about logging

Logging as supplied by the python standard library can be used.

We make use of the standard logging levels DEBUG, INFO, WARNING, ERROR, CRITICAL.

In [1]:
import logging
import sys
from pathlib import Path

import numpy as np
import pandas as pd

import locan as lc

In [2]:
lc.show_versions(dependencies=False, verbose=False)


Locan:
   version: 0.7.dev7+gb690986

Python:
   version: 3.8.8

System:
python-bits: 64
    system: Windows
   release: 10
   version: 10.0.18362
   machine: AMD64
 processor: Intel64 Family 6 Model 142 Stepping 10, GenuineIntel
 byteorder: little
    LC_ALL: None
      LANG: None
    LOCALE: {'language-code': 'de_DE', 'encoding': 'cp1252'}


## Activate logging

In any script or notebook logging has to be enabled e.g. for streaming to stdout.

For changing the configuration logging has to be reloaded or the kernel be restarted.

In [3]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

A top-level logger has to be instantiated to process any log messages from the library.

In [4]:
logger = logging.getLogger()
logger

<RootLogger root (INFO)>

Further log messages can be added:

In [5]:
logger.info("test")

2021-03-20 08:04:58,607 - root - INFO - test


### Handling locan.logger

To change the filter level of locan log records, use an instance of the locan logger identified by its module name.

In [6]:
locan_logger = logging.getLogger('locan')
locan_logger.setLevel(logging.INFO)
locan_logger

<Logger locan (INFO)>

Changing the level of the locan logger to `logging.WARN` or higher, will switch off most locan log records.

## Logging in locan

In [7]:
dat = lc.simulate_tracks(n_walks=5, n_steps=100, ranges=((0,1000),(0,1000)),
                      diffusion_constant=1, seed=1)

dat.print_meta()

identifier: "1"
creation_date: "2021-03-20 08:05:03 +0100"
source: SIMULATION
state: RAW
history {
  name: "simulate_tracks"
  parameter: "{\'n_steps\': 100, \'ranges\': ((0, 1000), (0, 1000)), \'diffusion_constant\': 1, \'time_step\': 10, \'seed\': 1, \'n_walks\': 5}"
}
element_count: 500
frame_count: 100



In [8]:
lp = lc.LocalizationPrecision().compute(locdata=lc.LocData())

2021-03-20 08:05:04,758 - locan.analysis.localization_precision - WARNING - Locdata is empty.


In [9]:
lp = lc.LocalizationPrecision().compute(dat)

Processed frames:: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 98/98 [00:01<00:00, 56.73it/s]


### Logging in a pipeline

In [10]:
def computation(self, file):
    logger.info(f'computation started for file: {file}')
    return self

In [11]:
pipes = [lc.Pipeline(computation=computation, file=file).compute() for file in range(3)]

2021-03-20 08:05:06,639 - root - INFO - computation started for file: 0
2021-03-20 08:05:06,641 - root - INFO - computation started for file: 1
2021-03-20 08:05:06,643 - root - INFO - computation started for file: 2


Another example how to use logging in analysis pipelines is given by the `computation_test` function.

In [12]:
pipes = [lc.Pipeline(computation=lc.analysis.pipeline.computation_test, locdata=file).compute() for file in range(3)]

2021-03-20 08:05:06,652 - locan.analysis.pipeline - INFO - computation finished for locdata: 0
2021-03-20 08:05:06,654 - locan.analysis.pipeline - WARNING - An exception occurred for locdata: 0
2021-03-20 08:05:06,657 - locan.analysis.pipeline - INFO - computation finished for locdata: 1
2021-03-20 08:05:06,658 - locan.analysis.pipeline - WARNING - An exception occurred for locdata: 1
2021-03-20 08:05:06,660 - locan.analysis.pipeline - INFO - computation finished for locdata: 2
2021-03-20 08:05:06,663 - locan.analysis.pipeline - WARNING - An exception occurred for locdata: 2


In [13]:
print(pipes[0].computation_as_string())

def computation_test(self, locdata=None, parameter='test'):
    """ A pipeline definition for testing."""
    self.locdata = locdata
    something = 'changed_value'
    logger.debug(f'something has a : {something}')
    self.test = parameter
    logger.info(f'computation finished for locdata: {locdata}')

    try:
        raise NotImplementedError
    except NotImplementedError:
        logger.warning(f'An exception occurred for locdata: {locdata}')

    return self



### Logging in multiprocessing with ray

To enable logging in multiprocessing using ray you need to include a default configuration in the computation function: `logging.basicConfig(level=logging.INFO)`.

## Logging in locan - third party libraries

Some third-party libraries provide their own logging system. Typically the individual loggers can be imported and modified.

In [14]:
import trackpy as tr

In [15]:
tr.logger

<Logger trackpy (INFO)>

alternatively

In [16]:
trackpy_logger = logging.getLogger('trackpy')
trackpy_logger

<Logger trackpy (INFO)>

Depending on the library various methods can be used to change the logging level. All of the following can be used.

In [17]:
locdata_new, track_series = lc.track(dat, search_range=5)

Frame 99: 5 trajectories present.
